<img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/horizontal-primary-light.png" alt="he-black-box" width="600"/>


# Homomorphic Encryption using Duet: Data Scientist
## Tutorial 1: Logistic Regression over Encrypted Data



Welcome!
This tutorial will show you how to train and evaluate a Logistic Regression using Duet and TenSEAL. This notebook shows the Data Scientist view on the operations.


We recommend going through Tutorial 0 before trying this one.

## Setup

All modules are imported here, make sure everything is installed by running the cell below.

In [1]:
import syft as sy
import tenseal as ts
import torch
import pandas as pd
import random
import numpy as np
from syft.grid.client.client import connect
from syft.grid.client.grid_connection import GridHTTPConnection
from syft.core.node.domain.client import DomainClient
import pytest
from time import time
import matplotlib.pyplot as plt

sy.load("tenseal")


## Connect to PyGrid

Connect to PyGrid Domain server.

In [3]:
client = connect(
    url="http://localhost:5000", # Domain Address
    credentials={"email":"user@email.com", "password":"userpwd"},
    conn_type= GridHTTPConnection, # HTTP Connection Protocol
    client_type=DomainClient) # Domain Client type

### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 1 : Now STOP and run the Data Owner notebook until the next checkpoint.

### Define The Model

Here we define the models used by the Data Scientist.

 - __LR__ is a simple logistic regression for plain data.
 - __EncryptedLR__ adapts the forward method to the API exposed by the encrypted ciphertexts.

In [4]:
class LR(torch.nn.Module):

    def __init__(self, n_features):
        super(LR, self).__init__()
        self.lr = torch.nn.Linear(n_features, 1)
        
    def forward(self, x):
        out = torch.sigmoid(self.lr(x))
        return out
    
class EncryptedLR:
    def __init__(self, torch_lr):
        # TenSEAL processes lists and not torch tensors
        # so we take out parameters from the PyTorch model
        self.weight = torch_lr.lr.weight.data.tolist()[0]
        self.bias = torch_lr.lr.bias.data.tolist()
        
    def forward(self, enc_x):
        # We don't need to perform sigmoid as this model
        # will only be used for evaluation, and the label
        # can be deduced without applying sigmoid
        enc_out = enc_x.dot(self.weight) + self.bias
        return enc_out
    
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

### Request the training data

We request access to the training data.

In [5]:
client.store.pandas

,ID,Tags,Description,object_type
0,<UID: 90a2a838f5b149aeb39a62b8bbe2835e>,[x_train],,<class 'torch.Tensor'>
1,<UID: fb476d205c804da9a8d5bbff12140398>,[y_train],,<class 'torch.Tensor'>


In [6]:
x_train_ptr = client.store["x_train"]
y_train_ptr = client.store["y_train"]
x_train_ptr.request(reason="I would like to get the training data")
y_train_ptr.request(reason="I would like to get the training labels")


In [7]:
client.requests.pandas

,Requested Object's tags,Reason,Request ID,Requested Object's ID,Requested Object's type
0,[x_train],I would like to get the training data,<UID: 1660da134c444d29ae8a0e79f86d503d>,<UID: 90a2a838f5b149aeb39a62b8bbe2835e>,<class 'torch.Tensor'>
1,[y_train],I would like to get the training labels,<UID: c75a24e4030348bf95ba203821401784>,<UID: fb476d205c804da9a8d5bbff12140398>,<class 'torch.Tensor'>


### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 2 : Now STOP and run the Data Owner notebook until the next checkpoint.

### Train the logistic regression

Now we train our Logistic regression with the data from the DO instance.

In [8]:
x_train = x_train_ptr.get(delete_obj=False)
y_train = y_train_ptr.get(delete_obj=False)

n_features = x_train.shape[1]
model = LR(n_features)

# use gradient descent with a learning_rate=1
optim = torch.optim.SGD(model.parameters(), lr=1)

# use Binary Cross Entropy Loss
criterion = torch.nn.BCELoss()

In [9]:
EPOCHS = 20

def train(model, optim, criterion, x, y, epochs=EPOCHS):
    for e in range(1, epochs + 1):
        optim.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optim.step()
        print(f"Loss at epoch {e}: {loss.data}")
    return model

model = train(model, optim, criterion, x_train, y_train)

Loss at epoch 1: 0.7149590253829956
Loss at epoch 2: 0.6483930349349976
Loss at epoch 3: 0.6229013204574585
Loss at epoch 4: 0.6113530993461609
Loss at epoch 5: 0.605393648147583
Loss at epoch 6: 0.6020280718803406
Loss at epoch 7: 0.5999988317489624
Loss at epoch 8: 0.5987114310264587
Loss at epoch 9: 0.5978598594665527
Loss at epoch 10: 0.5972760915756226
Loss at epoch 11: 0.5968633890151978
Loss at epoch 12: 0.5965639352798462
Loss at epoch 13: 0.5963418483734131
Loss at epoch 14: 0.5961740612983704
Loss at epoch 15: 0.5960456132888794
Loss at epoch 16: 0.5959459543228149
Loss at epoch 17: 0.595867931842804
Loss at epoch 18: 0.5958065986633301
Loss at epoch 19: 0.5957579612731934
Loss at epoch 20: 0.5957191586494446


### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 3 : Now STOP and run the Data Owner notebook until the next checkpoint.

### Request the encrypted tensors

We create the model variant that cand handle encrypted tensors.

In [10]:
eelr = EncryptedLR(model)

### Check the Duet store

Now we can see all evaluating objects and get pointers to them.


__Note__: we cannot get these without permission.

In [11]:
client.store.pandas

,ID,Tags,Description,object_type
0,<UID: 90a2a838f5b149aeb39a62b8bbe2835e>,[x_train],,<class 'torch.Tensor'>
1,<UID: fb476d205c804da9a8d5bbff12140398>,[y_train],,<class 'torch.Tensor'>
2,<UID: 28666e88d3694db7bfa214f794f931f9>,[context],,<class 'tenseal.enc_context.Context'>
3,<UID: 5a0a3af5aee7477bb15ec01f55cd07e8>,[enc_x_test],,<class 'syft.lib.python.list.List'>


### Get pointers to the encrypted vectors and request permission to work on them.

In [12]:
ctx_ptr = client.store["context"]
enc_x_test_ptr = client.store["enc_x_test"]

ctx_ptr.request(reason="I would like to get the context")
enc_x_test_ptr.request(reason="I would like to get encrypted test set")

### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 4 : Now STOP and run the Data Owner notebook until the next checkpoint.

### Encrypted evaluation

In [13]:
def encrypted_evaluation(model, enc_x_test):
    results = []
    
    correct = 0
    for enc_x in enc_x_test:
        # encrypted evaluation
        results.append(model(enc_x))
    return results

enc_x_test = enc_x_test_ptr.get(delete_obj=False)
ctx = ctx_ptr.get(delete_obj=False)
for t in enc_x_test:
    t.link_context(ctx)

eval_res = encrypted_evaluation(eelr, enc_x_test)

In [15]:
result_eval_ptr = sy.lib.python.List(eval_res).send(client, pointable=True, tags=["result_eval"])

client.store.pandas

,ID,Tags,Description,object_type
0,<UID: 90a2a838f5b149aeb39a62b8bbe2835e>,[x_train],,<class 'torch.Tensor'>
1,<UID: fb476d205c804da9a8d5bbff12140398>,[y_train],,<class 'torch.Tensor'>
2,<UID: 28666e88d3694db7bfa214f794f931f9>,[context],,<class 'tenseal.enc_context.Context'>
3,<UID: 5a0a3af5aee7477bb15ec01f55cd07e8>,[enc_x_test],,<class 'syft.lib.python.list.List'>
4,<UID: 5588e9efb0344e249630e6c40a7b356f>,[result_eval],,<class 'syft.lib.python.list.List'>
5,<UID: da652bb4061944d196af946e5b34e082>,[result_eval],,<class 'syft.lib.python.list.List'>


### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 5 : Now STOP and run the Data Owner notebook until the next checkpoint.

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft and TenSEAL on GitHub

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)
- [Star TenSEAL](https://github.com/OpenMined/TenSEAL)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org). #lib_tenseal and #code_tenseal are the main channels for the TenSEAL project.

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)